## Install Libraries


In [1]:
%pip install "ray[tune]"
# %pip install torch torchvision torchaudio # Uncomment this to install PyTorch 2.0+ on ilab (required version to run this program)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 33.6 MB/s eta 0:00:00


## Import Libraries


In [2]:
from filelock import FileLock
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from torch.utils.data import random_split
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from collections import OrderedDict


## Data loaders


In [3]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform
        )

        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform
        )

    return trainset, testset


## Configurable neural network


In [4]:
def flatten(x, start_dim=1, end_dim=-1):
    return x.flatten(start_dim=start_dim, end_dim=end_dim)


In [5]:
# Will need to modify the parameters and the network depending on what you are experimenting with

C, H, W = 3, 32, 32
num_classes = 10

channel_1 = 16
channel_2 = 32
channel_3 = 64
pool_kernel_size = 2

class Net(nn.Module):
    def __init__(self, kernel_size_1, pad_size_1, kernel_size_2, pad_size_2, kernel_size_3, pad_size_3, fc_count_1, fc_count_2, drop):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, channel_1, kernel_size_1, padding=pad_size_1)
        self.norm1 = nn.BatchNorm2d(channel_1)
        self.pool1 = nn.MaxPool2d(pool_kernel_size)
        self.conv2 = nn.Conv2d(channel_1, channel_2, kernel_size_2, padding=pad_size_2)
        self.norm2 = nn.BatchNorm2d(channel_2)
        self.pool2 = nn.MaxPool2d(pool_kernel_size)
        self.conv3 = nn.Conv2d(channel_2, channel_3, kernel_size_3, padding=pad_size_3)
        self.norm3 = nn.BatchNorm2d(channel_3)
        self.pool3 = nn.MaxPool2d(pool_kernel_size)        

        H_after_conv1 = int(H + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        W_after_conv1 = int(W + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        H_after_pool1 = int((H_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool1 = int((W_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        H_after_conv2 = int(H_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        W_after_conv2 = int(W_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        H_after_pool2 = int((H_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool2 = int((W_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)  
        H_after_conv3 = int(H_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        W_after_conv3 = int(W_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        H_after_pool3 = int((H_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool3 = int((W_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)

        self.fc1 = nn.Linear(H_after_pool3*W_after_pool3*channel_3, fc_count_1)
        if drop == 1:
          self.fc2 = nn.Linear(fc_count_1, fc_count_2)
          self.fc3 = nn.Linear(fc_count_2, 10)
        else:
          self.fc3 = nn.Linear(fc_count_1, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = flatten(x)
        x = self.fc1(x)
        #x = F.relu(x)
        if hasattr(self, 'drop'):
          x = self.fc2(x)
        #x = F.relu(x)
        x = self.fc3(x)

        return x



## The train function


In [6]:
def train_cifar(config):
    net = Net(
        config["kernel_size_1"], config["pad_size_1"], config["kernel_size_2"], config["pad_size_2"], 
        config["kernel_size_3"], config["pad_size_3"], config["fc_count_1"], config["fc_count_2"], config["drop"] 
    )  # Will need to modify the parameters depending on what you are experimenting with

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=1e-2)

    # Do not modify any code below!
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
            net.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=64, shuffle=True, num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=64, shuffle=True, num_workers=8
    )

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:
                print(
                    f"[{epoch + 1}, {i + 1:>5} loss: {running_loss / epoch_steps:.3f}]"
                )
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        os.makedirs("my_model", exist_ok=True)
        torch.save((net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report(
            {"loss": (val_loss / val_steps), "accuracy": correct / total},
            checkpoint=checkpoint,
        )
    print("Finished Training")


## Test set accuracy


In [7]:
def test_best_model(best_result):
    best_trained_model = Net(
        best_result.config["kernel_size_1"], best_result.config["pad_size_1"], best_result.config["kernel_size_2"], best_result.config["pad_size_2"], 
        best_result.config["kernel_size_3"], best_result.config["pad_size_3"], best_result.config["fc_count_1"], best_result.config["fc_count_2"], best_result.config["drop"]  
    )  # Will need to modify the parameters depending on what you are experimenting with

    # Do not modify any code below!
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(
        best_result.checkpoint.to_directory(), "checkpoint.pt"
    )

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Best trial test set accuracy: {correct / total}")


## Configuring the search space


In [8]:
# You will be experimenting with the hyperparameters here
# Use tune.grid_search to specify what values you want to experiment with a hyperparameter
config = {"kernel_size_1": tune.grid_search([3, 5]), 
          "pad_size_1": tune.grid_search([1, 2]), 
          "kernel_size_2": tune.grid_search([3, 5]), 
          "pad_size_2": tune.grid_search([1, 2]), 
          "kernel_size_3": tune.grid_search([5, 7]), 
          "pad_size_3": tune.grid_search([2, 3]), 
          "fc_count_1": tune.grid_search([512]),
          "fc_count_2": tune.grid_search([512]),
          "drop": tune.grid_search([0, 1])}



In [9]:
# Do not modify any code below!
def main(config):
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar),
            resources={"gpu": 1},
        ),
        param_space=config,
    )
    results = tuner.fit()

    best_result = results.get_best_result("accuracy", "max")

    print(f"Best trial config: {best_result.config}")
    print(f"Best trial final validation loss: {best_result.metrics['loss']}")
    print(f"Best trial final validation accuracy: {best_result.metrics['accuracy']}")

    test_best_model(best_result)

    return results.get_dataframe("accuracy", "max")


results_df = main(config)


2023-04-12 01:06:19,529	INFO worker.py:1553 -- Started a local Ray instance.


(train_cifar pid=3415) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_01-06-18/train_cifar_3c7b3_00000_0_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=3,kernel_size_3=5,pad_size_1=1,pad_si_2023-04-12_01-06-21/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:01<00:00, 87659110.12it/s]


(train_cifar pid=3415) Extracting /root/ray_results/train_cifar_2023-04-12_01-06-18/train_cifar_3c7b3_00000_0_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=3,kernel_size_3=5,pad_size_1=1,pad_si_2023-04-12_01-06-21/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_01-06-18/train_cifar_3c7b3_00000_0_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=3,kernel_size_3=5,pad_size_1=1,pad_si_2023-04-12_01-06-21/data
(train_cifar pid=3415) Files already downloaded and verified


Trial name,accuracy,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_cifar_3c7b3_00000,0.7136,2023-04-12_01-07-30,True,,1768784d84b54425b2171e9f5cfa3cd8,"0_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=3,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",39b3058fbaf7,10,0.842178,172.28.0.12,3415,True,65.9811,4.52254,65.9811,1681261650,0,,10,3c7b3_00000,0.00365257
train_cifar_3c7b3_00001,0.7141,2023-04-12_01-08-19,True,,1768784d84b54425b2171e9f5cfa3cd8,"1_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=3,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",39b3058fbaf7,10,0.832129,172.28.0.12,3415,True,49.1067,4.32279,49.1067,1681261699,0,,10,3c7b3_00001,0.00365257
train_cifar_3c7b3_00002,0.7015,2023-04-12_01-09-09,True,,1768784d84b54425b2171e9f5cfa3cd8,"2_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=5,kernel_size_2=3,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",39b3058fbaf7,10,0.871768,172.28.0.12,3415,True,50.2121,4.43022,50.2121,1681261749,0,,10,3c7b3_00002,0.00365257
train_cifar_3c7b3_00003,0.7142,2023-04-12_01-09-59,True,,1768784d84b54425b2171e9f5cfa3cd8,"3_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=5,kernel_size_2=3,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",39b3058fbaf7,10,0.834687,172.28.0.12,3415,True,49.6611,4.41022,49.6611,1681261799,0,,10,3c7b3_00003,0.00365257
train_cifar_3c7b3_00004,0.7115,2023-04-12_01-10-50,True,,1768784d84b54425b2171e9f5cfa3cd8,"4_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=5,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",39b3058fbaf7,10,0.844072,172.28.0.12,3415,True,50.5037,4.36895,50.5037,1681261850,0,,10,3c7b3_00004,0.00365257
train_cifar_3c7b3_00005,0.7193,2023-04-12_01-11-39,True,,1768784d84b54425b2171e9f5cfa3cd8,"5_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=5,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",39b3058fbaf7,10,0.829669,172.28.0.12,3415,True,49.2693,4.50753,49.2693,1681261899,0,,10,3c7b3_00005,0.00365257
train_cifar_3c7b3_00006,0.6986,2023-04-12_01-12-45,True,,1768784d84b54425b2171e9f5cfa3cd8,"6_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=5,kernel_size_2=5,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",39b3058fbaf7,10,0.876707,172.28.0.12,3415,True,66.457,4.37344,66.457,1681261965,0,,10,3c7b3_00006,0.00365257
train_cifar_3c7b3_00007,0.7101,2023-04-12_01-13-41,True,,1768784d84b54425b2171e9f5cfa3cd8,"7_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=5,kernel_size_2=5,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",39b3058fbaf7,10,0.861159,172.28.0.12,3415,True,55.2633,4.38082,55.2633,1681262021,0,,10,3c7b3_00007,0.00365257
train_cifar_3c7b3_00008,0.7146,2023-04-12_01-14-35,True,,1768784d84b54425b2171e9f5cfa3cd8,"8_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=3,kernel_size_3=7,pad_size_1=1,pad_size_2=1,pad_size_3=2",39b3058fbaf7,10,0.850743,172.28.0.12,3415,True,54.1084,4.4279,54.1084,1681262075,0,,10,3c7b3_00008,0.00365257
train_cifar_3c7b3_00009,0.7145,2023-04-12_01-15-24,True,,1768784d84b54425b2171e9f5cfa3cd8,"9_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=3,kernel_size_3=7,pad_size_1=1,pad_size_2=1,pad_size_3=2",39b3058fbaf7,10,0.852111,172.28.0.12,3415,True,49.3474,4.30919,49.3474,1681262124,0,,10,3c7b3_00009,0.00365257


...

(train_cifar pid=3415) Extracting /root/ray_results/train_cifar_2023-04-12_01-06-18/train_cifar_3c7b3_00127_127_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=5,kernel_size_2=5,kernel_size_3=7,pad_size_1=2,pad__2023-04-12_02-57-09/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_01-06-18/train_cifar_3c7b3_00127_127_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=5,kernel_size_2=5,kernel_size_3=7,pad_size_1=2,pad__2023-04-12_02-57-09/data
(train_cifar pid=3415) Files already downloaded and verified
(train_cifar pid=3415) Finished Training


2023-04-12 02:57:59,973	INFO tune.py:798 -- Total run time: 6699.19 seconds (6698.99 seconds for the tuning loop).


Best trial config: {'kernel_size_1': 3, 'pad_size_1': 1, 'kernel_size_2': 3, 'pad_size_2': 1, 'kernel_size_3': 5, 'pad_size_3': 3, 'fc_count_1': 512, 'fc_count_2': 512, 'drop': 0}
Best trial final validation loss: 0.7844796173132149
Best trial final validation accuracy: 0.7358


100%|██████████| 170498071/170498071 [00:02<00:00, 76257705.52it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Best trial test set accuracy: 0.685


In [10]:
final_df = results_df.drop(
    columns=[
        "should_checkpoint",
        "timesteps_total",
        "episodes_total",
        "training_iteration",
        "trial_id",
        "experiment_id",
        "date",
        "timestamp",
        "pid",
        "hostname",
        "node_ip",
        "time_since_restore",
        "timesteps_since_restore",
        "iterations_since_restore",
        "warmup_time",
        "logdir",
    ]
)
final_df.sort_values(by=['accuracy'], ascending=False).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/drop,config/fc_count_1,config/fc_count_2,config/kernel_size_1,config/kernel_size_2,config/kernel_size_3,config/pad_size_1,config/pad_size_2,config/pad_size_3
64,0.784480,0.7358,4.530466,False,51.226842,0,512,512,3,3,5,1,1,3
53,0.805207,0.7303,4.477792,False,52.509049,1,512,512,3,5,5,2,2,2
118,0.822190,0.7268,4.614029,False,60.857283,0,512,512,5,5,5,2,2,3
52,0.814539,0.7248,4.365737,False,50.575394,0,512,512,3,5,5,2,2,2
125,0.821893,0.7248,4.486585,False,53.138291,1,512,512,3,5,7,2,2,3
44,0.805492,0.7246,4.363969,False,49.559722,0,512,512,3,5,7,1,2,2
85,0.805252,0.7237,4.471916,False,47.607913,1,512,512,3,5,5,2,1,3
95,0.806845,0.7235,4.461617,False,47.618263,1,512,512,5,5,7,2,1,3
102,0.800505,0.7233,4.481373,False,47.415167,0,512,512,5,5,5,1,2,3
101,0.787675,0.7233,4.479582,False,47.362523,1,512,512,3,5,5,1,2,3


In [11]:
final_df.sort_values(by=['accuracy']).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/drop,config/fc_count_1,config/fc_count_2,config/kernel_size_1,config/kernel_size_2,config/kernel_size_3,config/pad_size_1,config/pad_size_2,config/pad_size_3
106,0.911813,0.6909,4.611735,False,51.897197,0,512,512,5,3,7,1,2,3
74,0.869207,0.6980,4.359572,False,45.642803,0,512,512,5,3,7,1,1,3
42,0.889507,0.6988,4.362053,False,49.779928,0,512,512,5,3,7,1,2,2
14,0.907483,0.6991,4.485105,False,50.426091,0,512,512,5,5,7,1,1,2
43,0.878006,0.7001,4.580968,False,51.003935,1,512,512,5,3,7,1,2,2
79,0.862592,0.7007,4.453932,False,46.230705,1,512,512,5,5,7,1,1,3
2,0.871768,0.7015,4.430223,False,50.212069,0,512,512,5,3,5,1,1,2
108,0.853285,0.7015,4.483678,False,42.128919,0,512,512,3,5,7,1,2,3
6,0.869526,0.7018,4.378392,False,62.083573,0,512,512,5,5,5,1,1,2
15,0.866527,0.7024,4.410214,False,42.288522,1,512,512,5,5,7,1,1,2
